<a href="https://colab.research.google.com/github/embiekhochiu/dijkstra-algo-explain-visual/blob/main/Dijkstra-algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt update
!sudo apt install libcairo2-dev ffmpeg \
    texlive texlive-latex-extra texlive-fonts-extra \
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev
!pip install manim
!pip install IPython --upgrade

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
10 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
l

In [ ]:
from manim import *

Manim Community v0.18.0

In [ ]:
# @title Point
class Point:
    def __init__(self,x,y,node,radius=0.1):


      # Tọa độ x, y, 0 của 1 điểm trêm Video
      self.dot=Dot([x,y,0],radius=radius)
      self.node=node

      # Nhãn của điểm đó đè thẳng lên điểm vừa tạo
      self.label=LabeledDot(MathTex(node, font_size=36, color=BLACK)).next_to(self.dot,0)

In [ ]:
# @title DijkstraGraph
class DijkstraGraph:
  """
    Lưu ý: lines được lưu theo pair (node, next) với giá trị của node nhỏ hơn giá trị của next
    Bài toán giả định là độ thị không có chiều (các điểm đều có thể di chuyển qua lại)
  """
  def __init__(self,set_edges,coordinate, direct=False):
    self.node=set()
    self.pair=dict()
    self.line=set()
    for a,b,w in set_edges:
      self.node.add(a)
      self.node.add(b)
      # Đổi thứ tự node a, b để a luôn nhỏ hơn b, tiện tra cứu
      if a > b and not direct:
        a, b = b, a
      self.line.add((a,b))

    # Chuyển set thành list
    self.node=[*self.node,] # Assume it from 0 -> n
    self.line=[*self.line,]

    # VGroup của điểm
    self.points = VGroup()

    # VGroup của nhãn
    self.labels = VGroup()
    for i in range(len(self.node)):
      point=Point(coordinate[i][0],coordinate[i][1],self.node[i])
      self.labels.add(point.label)
      self.points.add(point.dot)
    self.lines = VGroup()
    idx=0
    for node,next in self.line:
      self.lines.add(Line(self.points[node].get_center(),self.points[next].get_center()))
      self.pair[(node,next)]=idx
      idx+=1

  def items(self):
    return self.points,self.labels,self.lines,self.pair

In [ ]:
%%manim -qm -v WARNING CustomDiGraph

class CustomDiGraph(Scene):
    """
      Class chung để hiển thị các node và đường đi của các điểm
      Lưu ý:
        node -> next

      Cần bổ sung:
      - Các điểm sẽ được xét khi 1 node được chọn
      - Hàm effect nháy của 1 đường thẳng (có thể chỉnh sửa màu và thời gian)
      - Hàm effect khi một điểm được chọn
      - Effect so sánh
      - Bảng lưu trữ giá trị
      - ...

    """
    def make(self):
        """
          Khởi tạo ra các thành phần để lưu trữ đồ thị
        """
        # Lưu trữ 1 điểm đến các điểm khác
        self.store_edges=dict()

        # Lưu trữ các cạnh trên đồ thị
        self.set_edges=set()

        # Lưu trữ các đỉnh
        self.vertices=set()

    def graph_init(self):
        """
            Khởi tạo vẽ đồ thị, dịch đồ thị sang vị trí LEFT 2 lần
        """
        # self.O_points.shift(LEFT*2)
        # self.O_lines.shift(LEFT*2)
        # self.O_labels.shift(LEFT*2)

        self.O_lines.set_z_index(self.O_points.z_index - 1)
        self.play(Create(self.O_points))
        self.play(FadeIn(self.O_lines))
        self.play(Create(self.O_labels))


    def effect_line(self, line, color=RED, width=10): #line: Line Mobject
      """
        Hiệu ứng "phát sáng" một đường thẳng
      """
      self.play(
        line.animate.set_stroke_width(width).set_color(color),
      )

    def effect_multiple_line(self,lines,color=RED,width=10): #lines: List of Line Mobject
      """
        Hiệu ứng "phát sáng" nhiều đường thẳng đường thẳng
      """
      for line in lines:
        self.play(
          line.animate.set_stroke_width(width).set_color(color),run_time=0.6
        )

    def effect_multiple_point(self,points,color=RED,width=0.55): #lines: List of Line Mobject
      """
        Hiệu ứng "phát sáng" nhiều đường thẳng đường thẳng
      """
      for point in points:
        self.play(
          point.animate.set_width(width).set_color(color),run_time=0.6
        )

    def not_check(self,dot):
      """
        Đang thử nghiệm
        Phát sáng một điểm
      """

      self.play(
          dot.animate.set_stroke_width(3),
          dot.animate.set_stroke_color(RED),
          dot.animate.set_color(PINK)
      )
      self.wait()
      self.play(
          dot.animate.set_stroke_width(0),
          dot.animate.set_stroke_color(WHITE),
          dot.animate.set_color(WHITE)
      )

    def construct(self):
        """
          Hàm chính để chạy.
          Đầu tiên phải gọi hàm make để khởi tạo các attribute lưu trữ đồ thị

          Sau đó thêm các điểm bằng method `add_node` theo công thức (node, next, weight)

          Phần còn lại là khởi tạo và vẽ đồ thị
          -   `coordinate`: Tọa độ các điểm được khởi tạo sẽ được ở trên bản đồ
          -   `self.weights`: Cost để đến được điểm đấy
          -   `self.weights2`: Giống `self.weights`, nhưng ban đầu các giá trị được để = 0 để tiện cập nhật cho các điểm lần đầu xét đến
          -   `self.scores` : VGroup để hiện thị cost đến từng điểm
        """

        # Đồ thị không chiều
        self.direct=False

        # Khởi tạo các biến lưu trữ
        self.make()



        # Thêm các điểm vào đồ thị
        self.add_node(0,1,2)
        self.add_node(1,2,3)
        self.add_node(1,4,3)
        self.add_node(2,3,2)
        self.add_node(3,4,7)
        self.add_node(3,6,3)
        self.add_node(4,5,3)
        self.add_node(5,7,3)
        self.add_node(6,7,2)

        coordinate     = [(-5.83, 0.55), (-2.25, -0.5), (-1, 2.7), (0.2, 0.6),
                        (1.75, -1.5), (4.67, -2.3), (4.69, 1.5), (4.72, -0.87)]


        self.direction = [ DL, DL, DR, DR, DL]

        # Khởi tạo các khối lưu trữ cho mục đích tìm kiếm elements thuận tiện
        self.vertices = list(self.vertices)

        # Lưu lại vị trí từ 1 điểm đến tất cả các điểm còn lại
        self.paths=[]
        for _ in range(len(self.vertices)):
          self.paths.append([])

        # Tra thứ tự của cặp điểm tương ứng với list `edges`


        # Weight và đánh dấu các điểm đã qua (visited)
        self.weights = [np.inf for _ in range(len(self.vertices))]
        self.visited = [False for _ in range(len(self.vertices))]
        self.weights[0] = 0
        self.weights2 = [ValueTracker(0) for _ in range(len(self.vertices))]



        # Trả lại các Mobject bằng class DijkstraGraph
        self.O_points,self.O_labels,self.O_lines,self.pair = DijkstraGraph(self.set_edges,coordinate).items()

        # O_lines(self.pair[(2,3)]) => tra duoc duong 2-3

        # Tạo ảnh bản đồ
        map_image = ImageMobject("/nhatoitrg.png")
        zoom_factor = 1.65

        # Scale hình ảnh để phóng to nó lên toàn màn hình
        map_image.scale(zoom_factor)

        # Đặt hình ảnh ở trung tâm của màn hình
        map_image.move_to(ORIGIN)

        map_image.set_z_index(self.O_points.z_index - 100)
        self.add(map_image)
        self.wait(2)

        # Vẽ đồ thị
        self.graph_init()

         # # Hiển thị weight giữa các điểm
        self.point = VGroup()
        for a,b,w in self.set_edges:

            # Hiển thị giá trị của một điểm là trung điểm giữa 2 node của nó
            self.point.add(LabeledDot(Tex(str(w), font_size=24,color=BLACK)).next_to(self.O_points[a].get_center()/2+self.O_points[b].get_center()/2,0))
        self.play(FadeIn(self.point))
        self.wait(3)


    def add_node(self, a, b, w):

        if a not in self.store_edges:
          self.store_edges[a] = []
        self.store_edges[a].append((b, w))
        self.set_edges.add((a, b, w))
        self.vertices.add(a)
        self.vertices.add(b)

        # Nếu mà đồ thị 2 chiều
        if not self.direct:
          if b not in self.store_edges:
            self.store_edges[b] = []
          self.store_edges[b].append((a, w))
          self.set_edges.add((b, a, w))





In [ ]:
%%manim -qm -v WARNING CustomDiGraph2

class CustomDiGraph2(Scene):
    """
      Class chung để hiển thị các node và đường đi của các điểm
      Lưu ý:
        node -> next

      Cần bổ sung:
      - Các điểm sẽ được xét khi 1 node được chọn
      - Hàm effect nháy của 1 đường thẳng (có thể chỉnh sửa màu và thời gian)
      - Hàm effect khi một điểm được chọn
      - Effect so sánh
      - Bảng lưu trữ giá trị
      - ...

    """
    def make(self):
        """
          Khởi tạo ra các thành phần để lưu trữ đồ thị
        """
        # Lưu trữ 1 điểm đến các điểm khác
        self.store_edges=dict()

        # Lưu trữ các cạnh trên đồ thị
        self.set_edges=set()

        # Lưu trữ các đỉnh
        self.vertices=set()

    def graph_init(self):
        """
            Khởi tạo vẽ đồ thị, dịch đồ thị sang vị trí LEFT 2 lần
        """
        # self.O_points.shift(LEFT*2)
        # self.O_lines.shift(LEFT*2)
        # self.O_labels.shift(LEFT*2)

        self.O_lines.set_z_index(self.O_points.z_index - 1)
        self.play(Create(self.O_points))
        self.play(FadeIn(self.O_lines),self.O_lines.animate.set_stroke_width(5).set_color(RED))
        self.play(Create(self.O_labels))


    def effect_line(self, line, color=RED, width=10): #line: Line Mobject
      """
        Hiệu ứng "phát sáng" một đường thẳng
      """
      self.play(
        line.animate.set_stroke_width(width).set_color(color),
      )

    def effect_multiple_line(self,lines,color=RED,width=10): #lines: List of Line Mobject
      """
        Hiệu ứng "phát sáng" nhiều đường thẳng đường thẳng
      """
      for line in lines:
        self.play(
          line.animate.set_stroke_width(width).set_color(color),run_time=0.6
        )

    def effect_multiple_point(self,points,color=RED,width=0.55): #lines: List of Line Mobject
      """
        Hiệu ứng "phát sáng" nhiều đường thẳng đường thẳng
      """
      for point in points:
        self.play(
          point.animate.set_width(width).set_color(color),run_time=0.6
        )

    def not_check(self,dot):
      """
        Đang thử nghiệm
        Phát sáng một điểm
      """

      self.play(
          dot.animate.set_stroke_width(3),
          dot.animate.set_stroke_color(RED),
          dot.animate.set_color(PINK)
      )
      self.wait()
      self.play(
          dot.animate.set_stroke_width(0),
          dot.animate.set_stroke_color(WHITE),
          dot.animate.set_color(WHITE)
      )

    def construct(self):
        """
          Hàm chính để chạy.
          Đầu tiên phải gọi hàm make để khởi tạo các attribute lưu trữ đồ thị

          Sau đó thêm các điểm bằng method `add_node` theo công thức (node, next, weight)

          Phần còn lại là khởi tạo và vẽ đồ thị
          -   `coordinate`: Tọa độ các điểm được khởi tạo sẽ được ở trên bản đồ
          -   `self.weights`: Cost để đến được điểm đấy
          -   `self.weights2`: Giống `self.weights`, nhưng ban đầu các giá trị được để = 0 để tiện cập nhật cho các điểm lần đầu xét đến
          -   `self.scores` : VGroup để hiện thị cost đến từng điểm
        """

        # Đồ thị không chiều
        self.direct=False

        # Khởi tạo các biến lưu trữ
        self.make()



        # Thêm các điểm vào đồ thị
        self.add_node(0,1,2)
        self.add_node(0,2,3)
        self.add_node(0,3,3)
        self.add_node(0,4,2)
        self.add_node(1,2,7)
        self.add_node(1,3,3)
        self.add_node(1,4,3)
        self.add_node(1,5,3)
        self.add_node(1,6,2)
        self.add_node(1,7,2)
        self.add_node(2,3,3)
        self.add_node(2,5,3)
        self.add_node(2,6,2)
        self.add_node(2,7,7)
        self.add_node(3,4,3)
        self.add_node(3,5,3)
        self.add_node(3,6,3)
        self.add_node(3,7,2)
        self.add_node(3,8,3)
        self.add_node(3,9,2)
        self.add_node(4,5,7)
        self.add_node(4,6,3)
        self.add_node(4,7,3)
        self.add_node(4,8,3)
        self.add_node(4,9,2)
        self.add_node(5,6,3)
        self.add_node(5,7,3)
        self.add_node(5,8,3)
        self.add_node(5,9,2)
        self.add_node(6,7,3)
        self.add_node(6,8,3)
        self.add_node(6,9,2)
        self.add_node(7,8,2)
        self.add_node(7,9,2)
        coordinate     = [(-4.55, 1.12), (-0.67, -1.5), (0.2, -2), (0.14, -0.27), (0.22, 1.27),
                        (1.57, 0), (2.67, -1.43), (2.76, 0.45), (5.08, 2.37), (5.75, 2.55)]


        self.direction = [ DL, DL, DR, DR, DL]

        # Khởi tạo các khối lưu trữ cho mục đích tìm kiếm elements thuận tiện
        self.vertices = list(self.vertices)

        # Lưu lại vị trí từ 1 điểm đến tất cả các điểm còn lại
        self.paths=[]
        for _ in range(len(self.vertices)):
          self.paths.append([])

        # Tra thứ tự của cặp điểm tương ứng với list `edges`


        # Weight và đánh dấu các điểm đã qua (visited)
        self.weights = [np.inf for _ in range(len(self.vertices))]
        self.visited = [False for _ in range(len(self.vertices))]
        self.weights[0] = 0
        self.weights2 = [ValueTracker(0) for _ in range(len(self.vertices))]



        # Trả lại các Mobject bằng class DijkstraGraph
        self.O_points,self.O_labels,self.O_lines,self.pair = DijkstraGraph(self.set_edges,coordinate).items()

        # O_lines(self.pair[(2,3)]) => tra duoc duong 2-3

        # Tạo ảnh bản đồ
        map_image = ImageMobject("/VietNamthegioi.png")
        zoom_factor = 1.65

        # Scale hình ảnh để phóng to nó lên toàn màn hình
        map_image.scale(zoom_factor)

        # Đặt hình ảnh ở trung tâm của màn hình
        map_image.move_to(ORIGIN)

        map_image.set_z_index(self.O_points.z_index - 100)
        self.add(map_image)
        self.wait(2)

        # Vẽ đồ thị
        self.graph_init()
        self.wait(3)


    def add_node(self, a, b, w):

        if a not in self.store_edges:
          self.store_edges[a] = []
        self.store_edges[a].append((b, w))
        self.set_edges.add((a, b, w))
        self.vertices.add(a)
        self.vertices.add(b)

        # Nếu mà đồ thị 2 chiều
        if not self.direct:
          if b not in self.store_edges:
            self.store_edges[b] = []
          self.store_edges[b].append((a, w))
          self.set_edges.add((b, a, w))



